In [44]:
import pandas as pd
import numpy as np
import json
import os

In [64]:
root = "."

# Load cleaned assays
assays_cleaned = pd.read_csv(os.path.join(root, "..", "output", "abaumannii", "assays_cleaned.csv"))

# Load clustering infomation
clusters = pd.read_csv(os.path.join(root, "..", "output", "abaumannii", "assays_clusters.csv"))

# Define PATH to parameters
PATH_TO_PARAMETERS = os.path.join(root, "..", "output", "abaumannii", 'parameters')

# Check that the order is the same
a = clusters[['assay_id', 'activity_type', 'unit', 'activities', 'cpds']]
b = assays_cleaned[['assay_id', 'activity_type', 'unit', 'activities', 'cpds']]
mask = (a == b) | (a.isna() & b.isna())
assert np.all(mask.to_numpy())
del a, b, mask

# Append clustering information
assays_cleaned['clusters_0.3'] = clusters['clusters_0.3']
assays_cleaned['clusters_0.6'] = clusters['clusters_0.6']
assays_cleaned['clusters_0.85'] = clusters['clusters_0.85']
# del clusters

In [80]:
ORGNISM_CURATED, TARGET_TYPE_CURATED, STRAIN, MUTATIONS, KDR, MEDIA = [], [], [], [], [], []

# Iterating over assays
for assay_id, activity_type, unit in assays_cleaned[['assay_id', 'activity_type', 'unit']].values:

    # Prepare filename
    filename = "_".join([str(assay_id), str(activity_type), str(unit), 'parameters']) + ".json"
    
    # Read JSON file
    with open(os.path.join(PATH_TO_PARAMETERS, filename), "r") as file:
        par = json.load(file)

    # Store results
    ORGNISM_CURATED.append(par['organism'])
    TARGET_TYPE_CURATED.append(par['target_type'])
    STRAIN.append(par['strain'])
    MUTATIONS.append(";".join(par['mutations']))
    KDR.append(";".join(par['known_drug_resistances']))
    MEDIA.append(par['media'])

# Complete table
assays_cleaned['organism_curated'] = ORGNISM_CURATED
assays_cleaned['target_type_curated'] = TARGET_TYPE_CURATED
assays_cleaned['strain'] = STRAIN
assays_cleaned['mutations'] = MUTATIONS
assays_cleaned['known_drug_resistances'] = KDR
assays_cleaned['media'] = MEDIA


In [81]:
assays_cleaned

,assay_id,assay_type,assay_organism,doc_chembl_id,target_type,target_chembl_id,target_organism,activity_type,unit,canonical_unit,...,standard_text_count,clusters_0.3,clusters_0.6,clusters_0.85,organism_curated,target_type_curated,strain,mutations,known_drug_resistances,media
0,CHEMBL4296188,F,Acinetobacter baumannii,CHEMBL4296181,ORGANISM,CHEMBL614425,Acinetobacter baumannii,INHIBITION,%,%,...,0,7338,18809,21357,Acinetobacter baumannii,ORGANISM,ATCC 19606,,,CAMBH media
1,CHEMBL4296193,F,Acinetobacter baumannii,CHEMBL4296181,ORGANISM,CHEMBL614425,Acinetobacter baumannii,INHIBITION,%,%,...,0,1085,1509,1564,Acinetobacter baumannii,ORGANISM,ATCC 19606,,,CAMBH media
2,CHEMBL4296188,F,Acinetobacter baumannii,CHEMBL4296181,ORGANISM,CHEMBL614425,Acinetobacter baumannii,MIC,umol.L-1,umol.L-1,...,0,670,930,1030,Acinetobacter baumannii,ORGANISM,ATCC 19606,,,CAMBH media
3,CHEMBL1640427,F,Acinetobacter baumannii,CHEMBL1629439,ORGANISM,CHEMBL614425,Acinetobacter baumannii,MIC,umol.L-1,umol.L-1,...,0,11,53,132,Acinetobacter baumannii,ORGANISM,,,,
4,CHEMBL2329976,F,Acinetobacter baumannii,CHEMBL2321876,ORGANISM,CHEMBL614425,Acinetobacter baumannii,ACTIVITY,NaN,NaN,...,0,1,5,74,Acinetobacter baumannii,ORGANISM,,,,iron-deficient M9 minimal media
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,CHEMBL4257928,F,Acinetobacter baumannii,CHEMBL4257451,ORGANISM,CHEMBL614425,Acinetobacter baumannii,MIC,umol.L-1,umol.L-1,...,0,1,1,1,Acinetobacter baumannii,ORGANISM,,,,broth microdilution
3003,CHEMBL4257919,A,Acinetobacter baumannii,CHEMBL4257451,ORGANISM,CHEMBL614425,Acinetobacter baumannii,MIC,umol.L-1,umol.L-1,...,0,1,1,1,Acinetobacter baumannii,ORGANISM,isolate 4,,,
3004,CHEMBL4257918,A,Acinetobacter baumannii,CHEMBL4257451,ORGANISM,CHEMBL614425,Acinetobacter baumannii,MIC,umol.L-1,umol.L-1,...,0,1,1,1,Acinetobacter baumannii,ORGANISM,isolate 3,,,
3005,CHEMBL4257917,A,Acinetobacter baumannii,CHEMBL4257451,ORGANISM,CHEMBL614425,Acinetobacter baumannii,MIC,umol.L-1,umol.L-1,...,0,1,1,1,Acinetobacter baumannii,ORGANISM,isolate 2,,,broth
